### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [ ]:
hparams.pro_type_file is None

In [ ]:
from modelModule.model5 import VAE5
import torch
model = VAE5(dim=16, pro_types=pro_types)

### 得到数据集相关属性

In [ ]:
import pickle
import pandas as pd
data_pd = pd.read_csv('Letter_train.csv')
col_names = list(data_pd)
# df[col_name].value_counts()
pro_types = [('discrete',16) for _ in col_names]
# with open('Letter_pro_type.pkl', 'wb') as f:
#     pickle.dump(pro_types, f)

In [ ]:
a = 0
a is not None

In [ ]:
import pickle
pro_types = pickle.load(open('Letter_pro_type.pkl', 'rb'))
pro_types

### dataloader验证

In [ ]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [ ]:
dataset = FlatDataset('Letter_train.csv', 
                    pro_type_file='Letter_pro_type.pkl')


In [ ]:
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
batch['normal_data']

In [ ]:
batch["miss_data"].long()

In [ ]:
batch["src_data"]

In [ ]:
test_data = imputed_data * global_max + global_min # 广播效应

In [ ]:
a = torch.randint(0,10, size=(5,4,3)).float()
print(a)
f = torch.nn.Softmax(dim=1)
f(a)


In [ ]:
a = torch.randint(0,10,size=(5,12,4)).float()
print(a)
model = torch.nn.Linear(4,10)
model(a[:,1,:]).shape

In [ ]:
a[:,0].reshape(-1,1)

In [ ]:
model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1).shape

In [ ]:
isinstance(model, torch.nn.Embedding)

In [ ]:
a = torch.randint(0,2,size=(5, 3))
b = torch.nn.Parameter(torch.randn(1, 4))
print(b)
print(a)

miss_matric = a.unsqueeze (-1).expand(5,3,4)
print(miss_matric)
miss_matric*b

In [ ]:
a = torch.randint(0,10,size=(5,3)).float()
model = torch.nn.Conv1d(in_channels=1, out_channels=8, kernel_size=1, stride=1)
print(a)
embedding_out = torch.tensor([])
embedding_out = torch.cat((embedding_out, model(a[:,0].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
embedding_out = torch.cat((embedding_out, model(a[:,1].reshape(-1,1,1)).permute(0, 2, 1)), dim = 1)
print(embedding_out.shape)
embedding_out

In [ ]:
b.shape

### 模型5精度测定

In [1]:
from dataModule.dataset1 import FlatDataset
# from modelModule.model1 import VAE
# from modelModule.model2 import VAE2
# from modelModule.model4 import VAE4
from modelModule.model5 import VAE5
from torch.utils.data import DataLoader
from utils import result_show, get_missing
import torch
import numpy as np
import pickle

In [2]:
## 模型参数载入
model = VAE5(dim=16, pro_types=pickle.load(open('Letter_pro_type.pkl', 'rb'))) # nhead=3
checkpoint_path = r'模型参数保存/Letter_model5_norm_way_minmax_norm_/version_1/checkpoints/epoch=124-step=7875.ckpt'    
data_norm = 'minmax_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
train_csv = r'Letter_train.csv'
test_csv = r'Letter_test.csv'

checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
## 载入训练集的均值(最小)方差(最大)
train_dataset = FlatDataset(train_csv, data_norm=data_norm, pro_type_file='Letter_pro_type.pkl')
model.get_global_min_max(dataset=train_dataset)
print('载入成功')

载入成功


In [3]:
test_dataset = FlatDataset(test_csv, data_norm=data_norm, pro_type_file='Letter_pro_type.pkl')
data_size = 10
src_data = np.stack([test_dataset[index]['src_data'] for index in range(data_size)], axis=0)
normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(data_size)], axis=0)
portion_normal_data = np.stack([test_dataset[index]['portion_normal_data'] for index in range(data_size)], axis=0)
_, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
input = portion_normal_data.copy()
input[miss_matrix == 0] = np.nan

In [4]:
imputed_data, output = model.inference(input, miss_matrix) # inference输出的结果已经重构后的结果
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()

In [ ]:
imputed_data[2]

In [ ]:
imputed_data[1]

In [5]:
result_show(src_data, imputed_data, miss_matrix)

原始数据 [ 3.  2.  7. 11.  5.  1.  3.  6.  3.  2.  4.  4.  5.  2.  3.  5.  7. 12.
  4.  5.  7.  7.  9.  3.  8. 10. 10.  5.  6.  7.  6.  4.  2.  8.  2.  6.
  8.  7.  5.  7.  8.  7.  5. 11.  5.  8.  7.  8.  9.  7.  4.  9.  6.  0.
  8.  8.]
插补数据 [ 3.76715899  9.73495603  6.96657121 10.19531339 10.31874776 11.45070791
  8.80206048  7.10702375  9.16053593  8.39060247  6.18419588  5.85392877
 12.01961964  6.08233452  4.32285279  9.01717633  6.28888741  5.50018877
  6.94242507  3.91926527  1.87348466  6.23711318  7.74271667  5.91959551
  6.51791856  3.71302642  8.02937597  1.21733107  3.5483712   3.95827398
  4.78866935  1.25858858 10.4246968   8.64836991  2.07482211  3.66708733
  8.6186716   1.77778546  1.44823663 11.59407467 11.76233143  4.41120341
  4.52153385  6.80344597  6.79034561  7.32864037  3.20636719 11.74223632
  5.39332911  1.54906187  8.82945657 10.22236794  5.03238782  8.17226082
  4.74971563  6.08671725]


array([2.55719661e-01, 3.86747799e+00, 4.77554116e-03, 7.31533277e-02,
       1.06374955e+00, 1.04507078e+01, 1.93402016e+00, 1.84503957e-01,
       2.05351197e+00, 3.19530122e+00, 5.46048968e-01, 4.63482192e-01,
       1.40392393e+00, 2.04116725e+00, 4.40950929e-01, 8.03435264e-01,
       1.01587513e-01, 5.41650936e-01, 7.35606266e-01, 2.16146946e-01,
       7.32359333e-01, 1.08983832e-01, 1.39698148e-01, 9.73198500e-01,
       1.85260180e-01, 6.28697357e-01, 1.97062403e-01, 7.56533785e-01,
       4.08604800e-01, 4.34532288e-01, 2.01888442e-01, 6.85352853e-01,
       4.21234838e+00, 8.10462384e-02, 3.74110563e-02, 3.88818777e-01,
       7.73339494e-02, 7.46030648e-01, 7.10352673e-01, 6.56296380e-01,
       4.70291428e-01, 3.69828083e-01, 9.56932304e-02, 3.81504912e-01,
       3.58069121e-01, 8.39199534e-02, 5.41947543e-01, 4.67779539e-01,
       4.00741209e-01, 7.78705446e-01, 1.20736414e+00, 1.35818660e-01,
       1.61268696e-01, 8.17226082e+08, 4.06285546e-01, 2.39160344e-01])

In [ ]:
imputed_data

In [ ]:
imputed_data.shape

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))